In [ ]:
from datetime import date
import random
import time
import yfinance as yf
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

from numpy.fft import fft, ifft, fftshift
import numpy as np
from numpy import log, sqrt, exp
from numpy import pi, log, exp, real

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t, poisson
from scipy.linalg import solve_banded
from scipy.optimize import minimize, differential_evolution
from scipy.integrate import quad
from scipy.special import roots_laguerre
from scipy.interpolate import interp1d
from scipy.sparse import diags, kron, identity, csr_matrix, eye, csc_matrix
from scipy.sparse.linalg import spsolve
import scipy.sparse as sp
import scipy.sparse.linalg as spla
from scipy.interpolate import RegularGridInterpolator

import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

#import pymc as pm
#import arviz as az

from tensorflow import keras
#from tensorflow.keras.utils import plot_model

######################################
#from pmdarima import auto_arima
#from diptest import diptest

In [ ]:
class heston_options_value:
    def __init__(self, S0, K, T, r, rho, kappa, sigma, mu, theta, v0, option_type):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.kappa = kappa
        self.theta = theta
        self.sigma = sigma
        self.rho = rho
        self.v0 = v0
        self.mu = mu  # Not used in risk-neutral pricing
        self.option_type = option_type.lower()

    def compute_greeks_closed(self, h=1e-4):
        base_price = self.heston_p1p2()

        # ---- Delta: dPrice/dS0 ----
        S0_up = self.S0 + h
        S0_down = self.S0 - h
        self.S0 = S0_up
        price_up = self.heston_p1p2()
        self.S0 = S0_down
        price_down = self.heston_p1p2()
        delta = (price_up - price_down) / (2 * h)
        gamma = (price_up - 2 * base_price + price_down) / (h**2)
        self.S0 = S0_up - h  # Reset to original

        # ---- Vega: dPrice/dv0 ----
        v0_up = self.v0 + h
        v0_down = self.v0 - h
        self.v0 = v0_up
        price_up = self.heston_p1p2()
        self.v0 = v0_down
        price_down = self.heston_p1p2()
        vega = (price_up - price_down) / (2 * h)
        self.v0 = v0_up - h  # Reset

        # ---- Rho: dPrice/dr ----
        r_up = self.r + h
        r_down = self.r - h
        self.r = r_up
        price_up = self.heston_p1p2()
        self.r = r_down
        price_down = self.heston_p1p2()
        rho = (price_up - price_down) / (2 * h)
        self.r = r_up - h  # Reset

        # ---- Theta: -dPrice/dT ----
        T_up = self.T + h
        T_down = self.T - h
        self.T = T_up
        price_up = self.heston_p1p2()
        self.T = T_down
        price_down = self.heston_p1p2()
        theta = -(price_up - price_down) / (2 * h)
        self.T = T_up - h  # Reset

        return delta, gamma, theta, vega, rho

    def char_func(self, u):
        """
        Characteristic function for log(S_T) under the Heston model.
        """
        i = 1j

        x0 = np.log(self.S0)
        d = np.sqrt((self.rho * self.sigma * u * i - self.kappa)**2 + (self.sigma**2) * (u * i + u**2))
        g = (self.kappa - self.rho * self.sigma * u * i - d) / (self.kappa - self.rho * self.sigma * u * i + d)

        term1 = i * u * x0
        term2 = i * u *self.r * self.T + (self.kappa * self.theta / self.sigma**2) * ((self.kappa - self.rho * self.sigma * u * i - d) * self.T - 2 * np.log((1 - g * np.exp(-d * self.T)) / (1 - g)))
        term3 = (self.v0 / self.sigma**2) * (self.kappa - self.rho * self.sigma * u * i - d) * (1 - np.exp(-d * self.T)) / (1 - g * np.exp(-d * self.T))

        return np.exp(term1 + term2 + term3)

    def compute_P1_P2(self, N=4096, eta=0.25):
        u = np.arange(1, N+1) * eta  # Avoid u=0 to prevent division by zero
        logK = np.log(self.K)

        # Characteristic functions for P1 and P2
        phi_u = self.char_func(u - 1j) / self.char_func(-1j)  # For P1
        phi_v = self.char_func(u)                             # For P2

        integrand_P1 = np.real(np.exp(-1j * u * logK) * phi_u / (1j * u))
        integrand_P2 = np.real(np.exp(-1j * u * logK) * phi_v / (1j * u))

        # Trapezoidal rule (step size eta)
        P1 = 0.5 + (eta / np.pi) * np.sum(integrand_P1)
        P2 = 0.5 + (eta / np.pi) * np.sum(integrand_P2)

        return P1, P2

    def heston_p1p2(self, N=4096, eta=0.25):
        P1, P2 = self.compute_P1_P2(N, eta)

        if self.option_type == 'call':
            price = self.S0 * P1 - self.K * np.exp(-self.r * self.T) * P2
        else:
            price = self.K * np.exp(-self.r * self.T) * (1 - P2) - self.S0 * (1 - P1)

        return max(price, 0.0)  # ensure non-negativity

    ############################################################################################################

    def plot_option_value_K(self, logK_array, option_price, option_price_at_K, ax):
        # Main option price curve
        ax.plot(np.exp(logK_array), option_price, label='Option Price vs Strike', color='blue')

        # Highlight the option price at strike K
        ax.scatter([(self.K)], [option_price_at_K], color='red', marker='o', label=f'Price at K={self.K:.2f}: {option_price_at_K:.4f}', zorder=5)
        ax.axvline(x=(self.K), color='green', linestyle='--', label=f'Strike Price $K^o$')
        ax.set_xlabel('Strike Price (K)')
        ax.set_ylabel('Option Value')
        ax.set_title('Option Price vs Strike (Carr–Madan)')
        ax.set_xlim(0.5*self.K, 1.5*self.K)
        ax.set_ylim(-0.0001, self.K*0.5)
        #ax.legend()
        ax.grid(True)

    def plot_option_value_logK(self, logK_array, option_price, option_price_at_K, ax):
        # Main option price curve
        ax.plot(logK_array, option_price, label='Option Price vs Strike', color='blue')

        # Highlight the option price at strike K
        ax.scatter([np.log(self.K)], [option_price_at_K], color='red', marker='o', label=f'Price at K={self.K:.2f}: {option_price_at_K:.4f}', zorder=5)
        ax.axvline(x=np.log(self.K), color='green', linestyle='--', label=f'Strike Price $logK^o$')
        ax.set_xlabel('Strike Price (logK)')
        ax.set_ylabel('Option Value')
        ax.set_title('Option Price vs log Strike (Carr–Madan)')
        ax.set_xlim(0.5*np.log(self.K), 1.5*np.log(self.K))
        ax.set_ylim(-0.0001, self.K*0.5)
        #ax.legend()
        ax.grid(True)

    def chi_psi_vectorized(self, k, a, b, c, d):
        omega = k * np.pi / (b - a)      # omega_k
        delta = b - a                    # Precompute width
        # Arguments
        omega_c = omega * (c - a)
        omega_d = omega * (d - a)
        # Avoid division by zero for k=0 in psi
        omega_nonzero = omega.copy()
        omega_nonzero[0] = 1.0
        # ---- chi_k ----
        exp_c = np.exp(c)
        exp_d = np.exp(d)

        chi_k = (exp_d * (np.cos(omega_d) + omega * np.sin(omega_d)) - exp_c * (np.cos(omega_c) + omega * np.sin(omega_c)) )
        chi_k /= (1.0 + omega**2)

        # ---- psi_k ----
        psi_k = (np.sin(omega_d) - np.sin(omega_c)) / omega_nonzero

        # Correct psi_k[0] separately (limit as omega → 0)
        psi_k[0] = d - c

        return chi_k, psi_k

    def fft_cos(self, N=256, L=10):
        # Step 1: log-strike
        x0 = np.log(self.S0)

        # Step 2: Truncation range [a, b]
        c1 = x0 + self.r * self.T + (1 - np.exp(-self.kappa * self.T)) * (self.theta - self.v0) / (2 * self.kappa) - 0.5 * self.theta * self.T
        c2 = (1 / (8 * self.kappa**3)) * self.sigma**2 * self.T**2 * (1 - np.exp(-self.kappa * self.T))**2 * (self.v0 / self.T + self.theta)
        c4 = (3 * self.sigma**4 / (8 * self.kappa**3)) * (1 - np.exp(-self.kappa * self.T))**2 * (self.v0 / self.T + self.theta) #wang approximate

        a = c1 - L * np.sqrt(abs(c2) + np.sqrt(abs(c4)))
        b = c1 + L * np.sqrt(abs(c2) + np.sqrt(abs(c4)))

        k = np.arange(N)
        u = k * np.pi / (b - a)

        if self.option_type == 'call':
            c, d = np.log(self.K), b
            chi, psi = self.chi_psi_vectorized(k, a, b, c, d)
            Vk = (chi - self.K * psi)
        elif self.option_type == 'put':
            c, d = a, np.log(self.K)
            chi, psi = self.chi_psi_vectorized(k, a, b, c, d)
            Vk = (self.K * psi - chi)
        else:
            raise ValueError("Option type must be 'call' or 'put'.")

        Vk[0] *= 0.5  # k=0 term is halved

        # Step 4: COS formula
        phi = self.char_func(u)
        payoff = np.real( (2  / (b-a)) * phi * np.exp(-1j * u * a)) * Vk
        price = np.exp(-self.r * self.T) * np.sum(payoff)

        return price

    def fft_cm(self, alpha=1.5, N=4096, eta=0.25):
        # Frequency domain
        u = np.arange(N) * eta
        lambd = 2 * np.pi / (N * eta) #Nyquist

        # Compute log-strike grid
        k = np.arange(N) * lambd
        K_array = np.exp(k)

        # Dampened characteristic function
        phi = self.char_func(u - 1j * (alpha + 1))
        numerator = np.exp(-1j * u * k[0]) * np.exp(-self.r * self.T) * phi
        denominator = alpha**2 + alpha - u**2 + 1j * (2 * alpha + 1) * u
        integrand = numerator / denominator
        integrand *=  eta / np.pi

        # Apply FFT
        fft_output = np.fft.fft(integrand).real

        # Get option values
        call_prices = fft_output * np.exp(-alpha * k)

        if self.option_type == 'call':
            option_price = call_prices
            interp_fn = interp1d(K_array, option_price, kind='cubic', bounds_error=False, fill_value='extrapolate')
            option_price_at_K = interp_fn(self.K)
        elif self.option_type == 'put':
            option_price = call_prices - self.S0 + K_array * np.exp(-self.r * self.T)
            interp_fn = interp1d(K_array, option_price, kind='cubic', bounds_error=False, fill_value='extrapolate')
            option_price_at_K = interp_fn(self.K)

        return option_price_at_K, k, option_price, np.log(self.K), option_price_at_K


    ############################################################################################################
    def plot_option_value_contour(self, S, V, U, S_T=None, V_T=None, option_price_at_ST=None,
                                  ax=None, filled=True, levels=50,
                                  title="Option Price Contour Plot"):

        fig = None
        if ax is None:
            fig, ax = plt.subplots(figsize=(10, 7))

        if filled:
            contour = ax.contourf(S, V, U, levels=levels, cmap='viridis')
            fig.colorbar(contour, ax=ax, label='Option Price')
        else:
            contour = ax.contour(S, V, U, levels=levels, cmap='viridis')
            ax.clabel(contour, inline=True, fontsize=8)

        ax.axvline(x=self.K, color='red', linestyle='--', label='Strike Price K')
        if S_T is not None and V_T is not None:
            ax.plot(S_T, V_T, 'ro', label=f'Option Value: {option_price_at_ST:.4f}')
            ax.legend()

        ax.set_xlabel('Asset Price $S$')
        ax.set_ylabel('Variance $v$')
        ax.set_title(title)
        plt.tight_layout()
        plt.show()

    def plot_option_value(self, S, Vol,  V, S_T, Vol_T, option_price_at_ST, ax=None):
        fig = None
        if ax is None:
            fig, ax = plt.subplots(figsize=(10, 7))

        ax.plot(S, np.maximum(S - self.K, 0), '--k' , label='Option Value vs Stock Price at maturity')
        ax.plot(S, np.maximum(self.K - S, 0), '--k' , label='Option Value vs Stock Price at maturity')
        ax.plot(S, V, color = 'blue', label='Option Value vs Stock Price at present')
        ax.axvline(x=self.K, color='red', linestyle='--', label='Strike Price K')
        ax.axvline(x=S_T, color='green', linestyle='--', label=f'Stock Price $S_0$')
        ax.scatter([S_T], [option_price_at_ST], color='black', zorder=5, label=f'Option Value at $S_0$')

        ax.set_title('Option Value vs Stock Price at Maturity')
        ax.set_xlabel('Stock Price S')
        ax.set_ylabel('Option Value V')
        ax.set_xlim(self.K - 10, self.K + 10)
        ax.set_ylim(0, self.K)
        #ax.legend()
        ax.grid(True)


    def make_1D_diff_matrices(self, N, d):
        """Builds centered 1st and 2nd derivative matrices with Dirichlet BCs."""
        e = np.ones(N+1)
        D1 = sp.diags([-e, e], [-1, 1], shape=(N+1, N+1)) / (2 * d)
        D2 = sp.diags([e, -2*e, e], [-1, 0, 1], shape=(N+1, N+1)) / (d ** 2)
        D1 = D1.tolil(); D2 = D2.tolil()
        D1[0, :], D1[-1, :] = 0, 0
        D2[0, :], D2[-1, :] = 0, 0
        return D1.tocsr(), D2.tocsr()

    def build_L_heston(self, Nx, Nv, Dx, Dxx, Dv, Dvv, x, v, kappa, theta, sigma, rho, r):
        """
        Builds the full 2D operator L using Kronecker products for the Heston PDE.
        """

        # Term 1: 0.5 * v * d2/dx2
        L1 = sp.kron(Dxx, 0.5 * sp.diags(v))

        # Term 2: 0.5 * sigma^2 * v * d2/dv2
        L2 =sp.kron(sp.eye(Nx+1), 0.5 * sigma**2 * sp.diags(v) @ Dvv)

        # Term 3: (r - 0.5v) * du/dx
        L3 = sp.kron(Dx, sp.diags(r - 0.5 * v) )

        # Term 4: kappa*(theta - v) * du/dv
        L4 = sp.kron(sp.eye(Nx+1), sp.diags(kappa * (theta - v)) @ Dv)

        # Term 5: rho*sigma*v * d2/dx dv
        L5 = sp.kron(Dx, sp.diags(v)) @ sp.kron(sp.eye(Nx+1), rho * sigma * Dv)

        # Term 6: -r * u
        L6 = -r * sp.eye((Nx+1) * (Nv+1))

        L = L1 + L2 + L3 + L4 + L5 + L6
        return L.tocsr()


    def psor_cn_2pde(self, A, b, payoff, omega=1.2, tol=1e-2, max_iter=10):
        """
        Projected SOR method to solve (A u >= b, u >= payoff)
        """
        u = payoff.copy()
        N = len(b)

        for it in range(max_iter):
            u_old = u.copy()

            for i in range(N):
                row_start = A.indptr[i]
                row_end = A.indptr[i + 1]
                Ai = A.indices[row_start:row_end]
                Av = A.data[row_start:row_end]

                a_ii = 0.0
                sum_off_diag = 0.0

                for j, a_ij in zip(Ai, Av):
                    if j == i:
                        a_ii = a_ij  # diagonal
                    else:
                        sum_off_diag += a_ij * u[j]  # use current/old u[j] (Gauss-Seidel style)

                if a_ii == 0:
                    continue  # skip if no diagonal

                # Explicitly include (1 - ω)
                u_new = (1 - omega) * u[i] + (omega / a_ii) * (b[i] - sum_off_diag)
                u[i] = max(payoff[i], u_new)

            error = np.linalg.norm(u - u_old, ord=np.inf)
            if error < tol:
                break
#        else:
#            print("⚠️ PSOR did not converge within max_iter")

        return u

    def pde2_cn(self, exercise_type, exercise_days):
        # Grid parameters
        Nx, Nv, Nt = 80, 40, 100
        S_max, v_max = 3 * self.S0, 3 * self.v0
        x_min, x_max = np.log(1e-4), np.log(S_max)
        v_min = 1e-6

        dt = self.T / Nt

        x = np.linspace(x_min, x_max, Nx + 1)
        v = np.linspace(v_min, v_max, Nv + 1)
        dx = x[1] - x[0]
        dv = (v_max - v_min) / Nv

        X, V = np.meshgrid(x, v, indexing='ij')

        if exercise_type == 'european':
            exercise_binary = np.zeros(Nt + 1)
        elif exercise_type == 'american':
            exercise_binary = np.ones(Nt + 1)
        elif exercise_type == 'bermudan':
            exercise_binary = np.zeros(Nt + 1)
            for i in range(Nt+1):
                if i in exercise_days:
                    exercise_binary[i] = 1

        # Initial condition (European put)
        S = np.exp(X)

        # Terminal payoff
        if self.option_type == 'call':
            U = np.maximum(S - self.K, 0)
        elif self.option_type == 'put':
            U = np.maximum(self.K - S, 0)

        # Flatten U and prepare identity
        U_flat = U.flatten()
        I = sp.eye((Nx + 1) * (Nv + 1))

        Dx, Dxx = self.make_1D_diff_matrices(Nx, dx)
        Dv, Dvv = self.make_1D_diff_matrices(Nv, dv)

        # Build L once
        L = self.build_L_heston(Nx, Nv, Dx, Dxx, Dv, Dvv, x, v, self.kappa, self.theta, self.sigma, self.rho, self.r)

        # Time-stepping loop (Crank-Nicolson)
        A = I - 0.5 * dt * L
        B = I + 0.5 * dt * L

        for n in range(Nt):
            rhs = B @ U_flat

            # Solve A x = rhs
            if exercise_binary[n] == 1:
                payoff = np.maximum(S[:, None] - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - S[:, None], 0)
                payoff_flat = payoff.flatten()
                U_flat = self.psor_cn_2pde(A, rhs, payoff_flat)
            else:
                U_flat = spla.spsolve(A, rhs)

            # Reshape and plot
            U = U_flat.reshape((Nx + 1, Nv + 1))

            # Apply Dirichlet BCs (standard choice: payoff = 0 at boundaries)
            U[0, :] = 0        # x = x_min (S → 0)
            U[-1, :] = U[-1, :] = np.exp(x_max) - self.K if self.option_type == 'call' else 0  # x = x_max (S → ∞)
            U[:, 0] = U[:, 1]  # v = v_min (absorbing or Neumann: du/dv = 0)
            U[:, -1] = U[:, -2]  # v = v_max (same)

            U_flat = U.flatten()

        U = U_flat.reshape((Nx + 1, Nv + 1))

        # Interpolate to get value at (S0, v0)
        interp = RegularGridInterpolator((x, v), U)
        U_price = interp([[np.log(self.S0), self.v0]])[0]

        return U_price, S, V, U, self.S0, self.v0, U_price


###############################################################################################################################

    def psor_1D(self, A_diag, A_lower, A_upper, b, x0, lower_bound, omega=1.2, tol=1e-2, max_iter=10):
        x = x0.copy()
        N = len(x)
        inv_diag = 1.0 / A_diag

        for _ in range(max_iter):
            x_old = x.copy()
            for i in range(N):
                sigma = 0.0
                if i > 0:
                    sigma += A_lower[i] * x[i - 1]
                if i < N - 1:
                    sigma += A_upper[i] * x[i + 1]
                x[i] += omega * (b[i] - sigma - A_diag[i] * x[i]) * inv_diag[i]
                x[i] = max(lower_bound[i], x[i])
            if np.linalg.norm(x - x_old, np.inf) < tol:
                break
#        else:
#            print("\u26a0\ufe0f PSOR did not converge within max_iter")
        return x

    def make_derivative_matrices(self, N, d):
        e = np.ones(N)
        D2 = diags([e, -2 * e, e], offsets=[-1, 0, 1], shape=(N, N)).tocsc() / d**2
        D1 = diags([-e, e], offsets=[-1, 1], shape=(N, N)).tocsc() / (2 * d)
        return D1, D2

    def pde_cn_ADI_CS(self, exercise_type, exercise_days, cn_theta=0.5):
        Nx, Nv, N = 80, 40, 100
        S_max, v_max = 3 * self.S0, 3 * self.v0
        x_min, x_max = np.log(1e-4), np.log(S_max)
        v_min = 1e-4
        dt = self.T / N

        x = np.linspace(x_min, x_max, Nx + 1)
        v = np.linspace(v_min, v_max, Nv + 1)
        dx, dv = x[1] - x[0], v[1] - v[0]

        x_i = x[1:-1]
        v_j = v[1:-1]

        Nx1, Nv1 = Nx - 1, Nv - 1
        D1x, D2x = self.make_derivative_matrices(Nx1, dx)
        D1v, D2v = self.make_derivative_matrices(Nv1, dv)

        X, V = np.meshgrid(x, v, indexing='ij')
        S = np.exp(X)

        exercise_binary = np.zeros(N + 1)
        if exercise_type == 'american':
            exercise_binary[:] = 1
        elif exercise_type == 'bermudan':
            exercise_binary[exercise_days] = 1

        U0 = np.maximum(S - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - S, 0)
        payoff_lnS = np.maximum(np.exp(x[1:Nx]) - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - np.exp(x[1:Nx]), 0)

        for n in range(N):
            U = U0.copy()
            Ux = U[1:Nx, :]
            Uv = U[:, 1:Nv]

            Lx = 0.5 * (V[1:Nx, :] * (D2x @ Ux)) + (self.r - 0.5 * V[1:Nx, :]) * (D1x @ Ux)
            Lv = 0.5 * self.sigma**2 * (V[:, 1:Nv] * (Uv @ D2v.T)) + self.kappa * (self.theta - V[:, 1:Nv]) * (Uv @ D1v.T)
            Lxv = np.zeros_like(U[1:Nx, 1:Nv])

            RHS = U[1:Nx, 1:Nv] + dt * (Lx[:, 1:Nv] + Lv[1:Nx, :] + Lxv - self.r * U[1:Nx, 1:Nv])

            for j in range(1, Nv):
                vij = v[j]
                Nxi = Nx - 1
                drift = (self.r - 0.5 * vij)
                diffusion = 0.5 * vij
                alpha = cn_theta * dt * diffusion * np.ones(Nxi) / dx ** 2
                beta = cn_theta * dt * drift * np.ones(Nxi) / (2 * dx)

                A_lower = -alpha + beta
                A_main = 1 + 2 * alpha + self.r * dt * np.ones(Nxi)
                A_upper = -alpha - beta

                rhs = RHS[:, j - 1]
                guess = U[1:Nx, j]
                if exercise_binary[n] == 1:
                    guess = self.psor_1D(A_main, A_lower, A_upper, rhs, guess, payoff_lnS)
                else:
                    ab = np.zeros((3, Nxi))
                    ab[0, 1:] = A_upper[:-1]
                    ab[1, :] = A_main
                    ab[2, :-1] = A_lower[1:]
                    guess = solve_banded((1, 1), ab, rhs)
                U[1:Nx, j] = guess

            for jj in range(1, Nx):
                vij = v[1:-1]
                Nvi = Nv - 1
                drift = (self.r - 0.5 * vij)
                diffusion = 0.5 * vij
                alpha = cn_theta * dt * self.sigma ** 2 * vij / dv ** 2
                beta = cn_theta * dt * self.kappa * (self.theta - vij) / dv

                A_lower = -alpha + beta
                A_main = 1 + 2 * alpha + self.r * dt * np.ones(Nvi)
                A_upper = -alpha - beta

                rhs = RHS[jj - 1, :]
                guess = U[jj, 1:Nv]
                payoff = np.maximum(np.exp(x[jj]) - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - np.exp(x[jj]), 0)
                if exercise_binary[n] == 1:
                    guess = self.psor_1D(A_main, A_lower, A_upper, rhs, guess, payoff * np.ones(Nvi))
                else:
                    ab = np.zeros((3, Nvi))
                    ab[0, 1:] = A_upper[:-1]
                    ab[1, :] = A_main
                    ab[2, :-1] = A_lower[1:]
                    guess = solve_banded((1, 1), ab, rhs)
                U[jj, 1:Nv] = guess

            t_now = (n + 1) * dt
            if self.option_type == 'call':
                U[0, :] = 0
                U[-1, :] = S_max - self.K * np.exp(-self.r * (self.T - t_now))
            else:
                U[0, :] = self.K * np.exp(-self.r * (self.T - t_now))
                U[-1, :] = 0

            U[:, 0] = U[:, 1]
            U[:, -1] = U[:, -2]
            U0 = U.copy()

        interp = RegularGridInterpolator((x, v), U)
        U_price = interp([[np.log(self.S0), self.v0]])[0]

        return U_price, S, V, U, self.S0, self.v0, U_price


    #######################
    '''
    #WITHOUT using vectorization and sparse Matrices
    def psor_1D(self, A_diag, A_lower, A_upper, b, x0, lower_bound, omega=1.2, tol=1e-2, max_iter=10):
        x = x0.copy()
        N = len(x)

        # Precompute inverse diagonal for speed
        inv_diag = 1.0 / A_diag

        for _ in range(max_iter):
            x_old = x.copy()

            # Vectorized update loop
            for i in range(N):
                sigma = 0.0
                if i > 0:
                    sigma += A_lower[i] * x[i - 1]
                if i < N - 1:
                    sigma += A_upper[i] * x[i + 1]

                x[i] += omega * (b[i] - sigma - A_diag[i] * x[i]) * inv_diag[i]
                x[i] = max(lower_bound[i], x[i])

            # Early stopping condition
            if np.linalg.norm(x - x_old, np.inf) < tol:
                break
        else:
            print("\u26a0\ufe0f PSOR did not converge within max_iter")

        return x

    def pde_cn_ADI_CS(self, exercise_type, exercise_days, cn_theta =0.5):
        # Grid
        Nx = 80
        Nv = 40
        N = 100
        S_max =  3 * self.S0
        v_max =  3 * self.v0
        x_min = np.log(1e-4)
        x_max = np.log(S_max)
        v_min = 1e-4

        dt = self.T / N

        x = np.linspace(x_min, x_max, Nx + 1)
        v = np.linspace(v_min, v_max, Nv + 1)
        dx=x[1]-x[0]
        dv=v[1]-v[0]

        X, V = np.meshgrid(x, v, indexing='ij')
        S = np.exp(X)

        if exercise_type == 'european':
            exercise_binary = np.zeros(N + 1)
        elif exercise_type == 'american':
            exercise_binary = np.ones(N + 1)
        elif exercise_type == 'bermudan':
            exercise_binary = np.zeros(N + 1)
            for i in range(N+1):
                if i in exercise_days:
                    exercise_binary[i] = 1

        # Terminal payoff
        if self.option_type == 'call':
            U0 = np.maximum(S - self.K, 0)
        elif self.option_type == 'put':
            U0 = np.maximum(  self.K - S, 0)

        for n in range(N):

            U = U0.copy()
            # First derivatives and second derivatives
            d2Udx2 = (np.roll(U, -1, axis=0) - 2 * U + np.roll(U, 1, axis=0)) / dx**2
            dUdx   = (np.roll(U, -1, axis=0) - np.roll(U, 1, axis=0)) / (2 * dx)

            d2Udv2 = (np.roll(U, -1, axis=1) - 2 * U + np.roll(U, 1, axis=1)) / dv**2
            dUdv   = (np.roll(U, -1, axis=1) - np.roll(U, 1, axis=1)) / (2 * dv)

            d2Udxdv = (
                np.roll(np.roll(U, -1, axis=0), -1, axis=1) -
                np.roll(np.roll(U, -1, axis=0), 1, axis=1) -
                np.roll(np.roll(U, 1, axis=0), -1, axis=1) +
                np.roll(np.roll(U, 1, axis=0), 1, axis=1)
            ) / (4 * dx * dv)

            # Operators
            Lx  = 0.5 * V * d2Udx2 + (self.r - 0.5 * V) * dUdx
            Lv  = 0.5 * self.sigma**2 * V * d2Udv2 + self.kappa * (self.theta - V) * dUdv
            Lxv = self.rho * self.sigma * V * d2Udxdv

            # Full RHS of Crank-Nicolson (theta method)
            RHS = U + dt * (Lx + Lv + Lxv - self.r * U)

            # Step 1: Implicit in x (lnS), explicit in v
            for j in range(1, Nv):

                vij = v[j]
                ##############################################
                drift = (self.r - 0.5 * vij)
                diffusion = 0.5 * vij

                i = np.ones(Nx-1)
                alpha = cn_theta * dt * diffusion * i**2 / dx**2
                beta = cn_theta * dt * drift * i/ (2 * dx)
                #############################################
                # Finite difference coefficients
                A_lower = -alpha + beta
                A_main = 1 + 2 * alpha + self.r * dt
                A_upper = -alpha - beta

                rhs = RHS[1:Nx, j] + cn_theta * dt * (
                    -diffusion * (U0[2:Nx+1, j] - 2 * U0[1:Nx, j] + U0[0:Nx-1, j]) / dx**2 #second order derivative
                    -drift * (U0[2:Nx+1, j] - U0[0:Nx-1, j]) / (2 * dx) #First order derivative
                    + self.r * U0[1:Nx, j]
                )

                guess = U[1:Nx, j]
                # Apply PSOR if needed
                if exercise_binary[n] == 1:
                    payoff_1D = np.maximum(np.exp(x[1:Nx]) - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - np.exp(x[1:Nx]), 0)
                    guess = self.psor_1D(A_main, A_lower, A_upper, rhs, guess, payoff_1D)
                else:
                    Ax = np.zeros((3, Nx - 1))
                    Ax[0, 1:] = A_upper[:-1]
                    Ax[1, :] = A_main
                    Ax[2, :-1] = A_lower[1:]
                    guess = solve_banded((1, 1), Ax, rhs)
                U[1:Nx, j] = guess

            # Step 2: Implicit in v, explicit in x (lnS)
            for jj in range(1, Nx):  # Fixed x

                vij = v[1:-1]
                ##############################################
                drift = (self.r - 0.5 * vij)
                diffusion = 0.5 * vij

                alpha = cn_theta * dt * self.sigma**2 * vij / dv**2
                beta = cn_theta * dt * self.kappa * (self.theta - vij) / dv
                #############################################
                # Finite difference coefficients
                A_lower = -alpha + beta
                A_main = 1 + 2 * alpha + self.r * dt
                A_upper = -alpha - beta

                rhs = RHS[jj, 1:Nv] + cn_theta * dt * (
                    -0.5 * self.sigma**2 * vij * (U[jj, 2:Nv+1] - 2 * U[jj, 1:Nv] + U[jj, 0:Nv-1]) / dv**2
                    - self.kappa * (self.theta - vij) * (U[jj, 2:Nv+1] - U[jj, 0:Nv-1]) / (2 * dv)
                    + self.r * U0[jj, 1:Nv]
                )

                guess = U[jj, 1:Nv]
                # Apply PSOR if needed
                if exercise_binary[n] == 1:
                    Si = np.exp(x[jj])
                    payoff_1D = np.maximum(Si - self.K, 0) * np.ones(Nv - 1)  if self.option_type == 'call' else np.maximum(self.K - Si, 0) * np.ones(Nv - 1)
                    guess = self.psor_1D(A_main, A_lower, A_upper, rhs, guess, payoff_1D)
                else:
                    Av = np.zeros((3, Nv - 1))
                    Av[0, 1:] = A_upper[:-1]
                    Av[1, :] = A_main
                    Av[2, :-1] = A_lower[1:]
                    guess = solve_banded((1, 1), Av, rhs)
                U[jj, 1:Nv] = guess

            # === Apply Boundary Conditions ===
            t_now = (n + 1) * dt

            if self.option_type == 'call':
                U[0, :]  = 0  # Deep OTM call
                U[-1, :] = S_max - self.K * np.exp(-self.r * (self.T - t_now))  # Deep ITM call
            else:
                U[0, :]  = self.K * np.exp(-self.r * (self.T - t_now))
                U[-1, :] = 0

            U[:, 0]  = RHS[:, 1]    # Neumann at v = v_min
            U[:, -1] = RHS[:, -2]   # Neumann at v = v_max
            U0 = U.copy()

        # Interpolate to get value at (S0, v0)
        interp = RegularGridInterpolator((x, v), U)
        U_price = interp([[np.log(self.S0), self.v0]])[0]

        return U_price, S, V, U, self.S0, self.v0, U_price
        '''